In [3]:
import geopandas as gpd

import sys
sys.path.append('../Mask_RCNN')
import mrcnn.model as modellib

# Autoria
import canopy_detection_4b as orca

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy c-extensions failed.
- Try uninstalling and reinstalling numpy.
- If you have already done that, then:
  1. Check that you expected to use Python3.6 from "C:\Users\Colaborador\anaconda3\envs\rcnn\python.exe",
     and that you have no directories in your PATH or PYTHONPATH that can
     interfere with the Python and numpy version "1.17.5" you're trying to use.
  2. If (1) looks fine, you can open a new issue at
     https://github.com/numpy/numpy/issues.  Please include details on:
     - how you installed Python
     - how you installed numpy
     - your operating system
     - whether or not you have multiple versions of Python installed
     - if you built from source, your compiler versions and ideally a build log

- If you're working with a numpy git repository, try `git clean -xdf`
  (removes all files not under version control) and rebuild numpy.

Note: this error has many poss

In [ ]:
logs = '../Mask_RCNN/logs'
config = orca.OrangeCanopyConfig()
dataset = '../datasets/area_estudo_teste1/'

In [ ]:
model = modellib.MaskRCNN(mode='inference', config=config, model_dir=logs)

In [ ]:
weights = '../Mask_RCNN/logs/oranges_trees_canopy20211014T2316/mask_rcnn_oranges_trees_canopy_0060.h5'

model.load_weights(weights, by_name=True)

In [ ]:
img_test = gpd.read_file(dataset + '/img_grid.geojson')
ids = list(img_test.loc[img_test['split_samples'] == 'test'].index)
len(ids)

In [ ]:
results = orca.prediction(model, dataset, id_list=ids, verbose=0, first=False, join=True)

In [ ]:
results

In [ ]:
import os
import pandas as pd 
from tqdm import tqdm
from time import sleep

centroid_dist = 1
limiar_ovelap = 0.6

shape_path_aux = '../datasets/area_estudo_teste1/results/'

result_paths = []
for filename in os.listdir(shape_path_aux):
    if os.path.splitext(filename)[1].lower() == '.geojson':
        result_paths.append(os.path.join(shape_path_aux, filename))

pols = pd.concat([gpd.read_file(i) for i in result_paths], axis=0).reset_index(drop=True)
pols = pols[pols.geometry.is_valid]
pols['geometry'] = pols.buffer(0.1).buffer(-0.1).simplify(0.03)
pols = pols.explode('geometry').reset_index(drop=True)
pols['centroid'] = pols['geometry'].centroid
pols.insert(1, column='union', value=0)
pols.set_index(keys=pd.Index(range(1, pols.shape[0] + 1)), inplace=True)

with tqdm(total=len(pols.index)) as pbar:
    pbar.set_description("Simplificando - Macro")
    sleep(0.1)
    for i in pols.index:
        if pols.loc[i, 'union'] == 0:
            point = pols.loc[i, 'centroid']
            distance = pols['centroid'].distance(point)

            distance = distance[distance <= centroid_dist]

            if len(distance) > 1:
                pols.at[distance.index, 'union'] = i
            else:
                pols.at[i, 'union'] = i
        pbar.update(1)

# pols.drop('centroid', axis=1, inplace=True)
pols = pols.dissolve(by='union', as_index=False)
pols.drop(['union', 'centroid'], axis=1, inplace=True)
pols.drop(pols.loc[pols['geometry'].is_empty].index, axis=0, inplace=True)
pols.set_index(keys=pd.Index(range(1, pols.shape[0] + 1)), inplace=True)
pols['id'] = pols.index
# pols['centroid'] = pols['geometry'].centroid

# Gerar intersecoes entre os poligonos encontrados
# Uso de busca indexada (R-tree) com a função gpd.overlay()
over = gpd.overlay(pols, pols, how='intersection')
# Remover poligonos vazios (talvez possa ser dispensado a partir de agora)
over = over.loc[-over['geometry'].is_empty]
# Remover intersecoes originadas pela sobreposição da feição com ela mesma
over = over.loc[over['id_1'] != over['id_2']].explode().reset_index(drop=True)

delete_lines = []
# Barra de progresso #
with tqdm(total=len(over['id_1'])) as pbar:
    pbar.set_description("Simplificando - Micro")
    sleep(0.1)
    # Para cada poligono..
    for i in over['id_1']:
        # intersecções entre o poligono avaliado e todas as outras feições
        intersec = over[over['id_1'] == i]

        # Caso alguma área de intersecao avaliada seja maior que 60% (limiar_overlap) do proprio poligono avaliado
        # ou se poligono avaliado é 3,5x menor que aquele com o qual existe intersecao, o poligono será excluido
        if intersec.loc[intersec['geometry'].area >= limiar_ovelap * pols.loc[i, 'geometry'].area].any(axis=None) or \
                (3.5 * pols.loc[i, 'geometry'].area < pols.loc[intersec['id_2'], 'geometry'].area).any():
            delete_lines.append(i)

        pbar.update(1)

    # Identificar poligonos a serem apagados
    delete_lines = (list(set(delete_lines)))
# Fim da barra de progresso #

pols.drop(labels=delete_lines, axis=0, inplace=True)
pols = pols.explode().reset_index(drop=True)
pols.set_index(keys=pd.Index(range(1, pols.shape[0] + 1)), inplace=True)
pols['id'] = pols.index

In [ ]:
pols.to_file('../datasets/area_estudo_teste1/canopy_detection_result.geojson', driver='GeoJSON')

In [ ]:
all_files = [gpd.read_file(i) for i in result_paths]

In [ ]:
len(all_files)

In [ ]:
pols = pd.concat(all_files, axis=0).reset_index(drop=True)

In [ ]:
pols

In [ ]:
pols = pols[pols.geometry.is_valid]
pols

In [ ]:
pols.geometry.is_valid[-pols.geometry.is_valid]

In [ ]:
pols['geometry_n'] = pols.buffer(0.1).buffer(-0.1).simplify(0.03)

In [ ]:
pols['geometry_n'].is_empty.any()

In [ ]:
pols.geometry_n.is_empty[pols.geometry_n.is_empty]

In [ ]:
for i in range(len(all_files)):
    if all_files[i].is_empty.all():
        print(i)

In [ ]:
filtered_files = [gpd.read_file(i) for i in result_paths if gpd.read_file(i)]